# Behind the pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.6 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [13]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

In [14]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


In [18]:
print(outputs.hidden_states)

None


In [19]:
from transformers import AutoModelForSequenceClassification


checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [8]:
print(outputs.logits.shape)

torch.Size([2, 2])


None


In [9]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


In [10]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


In [11]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [37]:

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch

In [32]:
check_point="distilbert-base-uncased-finetuned-sst-2-english"

model=AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [49]:
input=[
       "what are your hobbies?",
    "hey How are you?",
    "Do you have sense or not?",
    "Is it even making sense to you?"

]

inputs=tokenizer(input, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  2054,  2024,  2115,  7570, 27982,  1029,   102,     0,     0],
        [  101,  4931,  2129,  2024,  2017,  1029,   102,     0,     0,     0],
        [  101,  2079,  2017,  2031,  3168,  2030,  2025,  1029,   102,     0],
        [  101,  2003,  2009,  2130,  2437,  3168,  2000,  2017,  1029,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [50]:
outputs=model(**inputs)

In [51]:
outputs.logits

tensor([[ 2.1283, -1.6736],
        [-2.0311,  2.2109],
        [ 3.2193, -2.6367],
        [ 3.5191, -2.8176]], grad_fn=<AddmmBackward0>)

In [52]:
predictions=torch.nn.functional.softmax(outputs.logits, dim=-1)



In [53]:
print(predictions)

tensor([[0.9782, 0.0218],
        [0.0142, 0.9858],
        [0.9971, 0.0029],
        [0.9982, 0.0018]], grad_fn=<SoftmaxBackward0>)


In [54]:
for i in predictions:
  print(i)

tensor([0.9782, 0.0218], grad_fn=<UnbindBackward0>)
tensor([0.0142, 0.9858], grad_fn=<UnbindBackward0>)
tensor([0.9971, 0.0029], grad_fn=<UnbindBackward0>)
tensor([0.9982, 0.0018], grad_fn=<UnbindBackward0>)


In [55]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

In [6]:
from transformers import AutoTokenizer

models=['pt', 'tf', 'np', 'jax', 'mlx']
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

encoded_input = tokenizer(["Hello, I'm a single sentence!", "hey how are you?"], truncation=True, padding=True, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[  101,  8667,   117,   146,   112,   182,   170,  1423,  5650,   106,
           102],
        [  101, 23998,  1293,  1132,  1128,   136,   102,     0,     0,     0,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [63]:
tokenizer.decode(encoded_input["input_ids"][1])

'[CLS] hey how are you? [SEP] [PAD] [PAD] [PAD] [PAD]'

In [8]:
import torch
model_inputs= torch.tensor(encoded_input.get("input_ids"))

/tmp/ipython-input-1984157988.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model_inputs= torch.tensor(encoded_input.get("input_ids"))


In [9]:
model_inputs

tensor([[  101,  8667,   117,   146,   112,   182,   170,  1423,  5650,   106,
           102],
        [  101, 23998,  1293,  1132,  1128,   136,   102,     0,     0,     0,
             0]])

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [11]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


In [12]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained("bert-base-cased")

In [15]:
tokens=tokenizer.tokenize("I’ve been waiting for a HuggingFace course my whole life.")
print(tokens)

ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)

decoded_string = tokenizer.decode(ids)
print(decoded_string)


['I', '’', 've', 'been', 'waiting', 'for', 'a', 'Hu', '##gging', '##F', '##ace', 'course', 'my', 'whole', 'life', '.']
[146, 787, 1396, 1151, 2613, 1111, 170, 20164, 10932, 2271, 7954, 1736, 1139, 2006, 1297, 119]
I ’ ve been waiting for a HuggingFace course my whole life.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
sequence = "I've been waiting for a HuggingFace course my whole life."


In [4]:
tokens=tokenizer.tokenize(sequence)
ids=tokenizer.convert_tokens_to_ids(tokens)
input_ids=torch.tensor(ids)

In [5]:
model(input_ids)

IndexError: too many indices for tensor of dimension 1

In [6]:
tokenized_input=tokenizer(sequence, return_tensors="pt")

print(tokenized_input)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [10]:
tokens=tokenizer.tokenize(sequence)
ids=tokenizer.convert_tokens_to_ids(tokens)
input_ids=torch.tensor([ids, ids])
print("input ids:", input_ids)
output=model(input_ids)
print("logits:", output.logits)

input ids: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
logits: tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [13]:
tokenizer.pad_token_type_id

0

In [18]:
tokenizer.pad_token_id

0

In [15]:
old_values=["I’ve been waiting for a HuggingFace course my whole life.", "I hate this so much!"]

In [16]:
sequence1=old_values[0]
sequence2=old_values[1]

tokens1=tokenizer.tokenize(sequence1)
tokens2=tokenizer.tokenize(sequence2)

ids1=tokenizer.convert_tokens_to_ids(tokens1)
ids2=tokenizer.convert_tokens_to_ids(tokens2)

print(len(ids1), len(ids2))

14 6


In [36]:
batched_ids=[
    ids1,
    ids2+[tokenizer.pad_token_id]*8
]
print(batched_ids)
attention_mask=[
    [1 for x in ids1],
    [1 for x in ids2]+[tokenizer.pad_token_id]*8
]
print(attention_mask)

print(len(batched_ids[0]), len(batched_ids[1]))
print(len(attention_mask[0]), len(attention_mask[1]))


[[1045, 1521, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012], [1045, 5223, 2023, 2061, 2172, 999, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]
14 14
14 14


In [37]:
input_ids=torch.tensor(batched_ids)
attention_masks=torch.tensor(attention_mask)

print(input_ids)
print(attention_masks)

tensor([[ 1045,  1521,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012],
        [ 1045,  5223,  2023,  2061,  2172,   999,     0,     0,     0,     0,
             0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


In [38]:
model(torch.tensor(batched_ids), attention_mask=attention_masks)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.5720,  2.6852],
        [ 3.1931, -2.6685]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
